To fix:
    dictionary is too small

In [1]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
#werds = pd.read_csv('/usr/share/dict/words')

In [3]:
word_list = []
with open("2of12id.txt") as f:
    s = f.readline()
    while (s):
        s_list = s.split()
        #print(s_list[0])
        if s_list[0][0].isalpha():
            word_list.append(s_list[0])
        s = f.readline()

In [4]:
words = pd.DataFrame(word_list).drop_duplicates()
words.columns=['word']

words['word'] = words['word'].apply(str.upper)
words['length'] = words['word'].apply(len)
words['used'] = False

words = words.sort_values('length').reset_index(drop=True)

words = words.reset_index()

In [5]:
words['word'] = words['word'].astype('str')

In [7]:
#read in list of common words, create length column, drop all words shorter than 5 letters

# common_words = pd.read_csv('common_words.csv', skiprows=1, names=['word'])
# common_words['word'] = common_words['word'].str.upper()
# common_words['length'] = common_words['word'].apply(len)
# common_words = common_words[common_words['length'] >= 5].sort_values('word')

In [8]:
black = '\033[30m'
grey = '\033[90m'
green = '\033[32m'
yellow = '\033[93m'

In [9]:
def pick_random_word(l):
    
    #get index of first word of that length
    beg = words[words.length == l]['index'].idxmin()
    end = words[words.length == l]['index'].idxmax() + 1
    
    index = random.randint(beg, end)
    
    while (words.iloc[index].used):
        index = random.randint(beg, end)
        
    words.iloc[index, 3] = True
    return words.iloc[index]['word']

In [10]:
def get_attempt(l):
    guess = input(black + 'Guess a ' + str(l) + '-character word:')
    
    if(len(guess) != l):
        return -1
    
    while (guess.upper() not in words['word'].values):
        guess = input(black + "That word isn't in our dictionary. Try again:")
    return guess

In [11]:
def evaluate_attempt(guess, word, alphabet):

    guess_colors = ['k'] * len(word)
    count = Counter(word)
    
    #first pass - determine green letters, decrement their counts
    for i in range(0, len(word)):
        if guess[i] == word[i]:
            guess_colors[i] = 'g'
            count[word[i]] -= 1
            
            
    for i in range(0, len(word)):
        #if letter is already green, skip
        if guess_colors[i] == 'g':
            continue
        elif guess[i] in count:
            if count[guess[i]] > 0:
                guess_colors[i] = 'y'
                count[guess[i]] -= 1
        else:
            if guess[i] in alphabet:
                alphabet.remove(guess[i])
                
    return guess_colors, alphabet

In [12]:
def display_attempts(guess_array, color_array, a, l):
    disp = ""
    for i in range(a+1):
        for j in range(l):
            if color_array[i,j] == 'y':
                disp += " " + yellow + guess_array[i,j] 
            elif color_array[i,j] == 'g':
                disp += " " + green + guess_array[i,j]
            else:
                disp += " " + black + guess_array[i,j]
        
        disp += "\n"
    
    return disp

In [13]:
def play():
    
    l = input("Choose Wordle length between 5 and 10:")
    
    if int(l) not in range(5, 11):
        l = input("Length must be integer between 5 and 10. Try again:")
    
    if int(l) not in range(5, 11):
        print("Must rerun game")
        return
    
    target_len = int(l)
    
    target = pick_random_word(target_len)
    
    target_arr = np.array(list(target.upper()))
    
    attempt = 0
    guesses = np.empty([6, len(target)], dtype=str)
    colors = np.empty([6, len(target)], dtype=str)
    alphabet = list(map(chr, range(65, 91)))
    
    while (attempt < 6):
        
        g = get_attempt(target_len)
        
        if (g == -1):
            print(black + "You can't follow directions!")
            return
            
        g = g.upper()
        
        c, alphabet = evaluate_attempt(g.upper(), target, alphabet)
        
        guesses[attempt] = list(g)
        colors[attempt] = c
        
        print(display_attempts(guesses, colors, attempt, target_len))
        
        if (g.upper() == target):
            print(black + "Congratulations! The wordle was " + target + "!")
            return
        
        alph = black + ""
        for char in alphabet:
            alph += " " + char
        print(alph)
        
        attempt += 1
        
    print(black + "Better luck next time, the wordle was " + target + "!")
    

In [14]:
play()

Choose Wordle length between 5 and 10:5
Guess a 5-character word:treat
 T R E A T

 B C D E F G H I J K L M N O P Q S U V W X Y Z
Guess a 5-character word:shine
 T R E A T
 S H I N E

 B C D E F G J K L M N O P Q U V W X Y Z
Guess a 5-character word:pouch
 T R E A T
 S H I N E
 P O U C H

 B D E F G J K L M N O Q V W X Y Z
Guess a 5-character word:money
 T R E A T
 S H I N E
 P O U C H
 M O N E Y

 B D E F G J K L M N O Q V W X Z
Guess a 5-character word:lemon
 T R E A T
 S H I N E
 P O U C H
 M O N E Y
 L E M O N

Congratulations! The wordle was LEMON!


In [ ]:
B_XO_